In [ ]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, random_split
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
!wget --no-check-certificate "https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5" -O SinglePhoton.hdf5
!wget --no-check-certificate "https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5" -O SingleElectron.hdf5

--2025-03-25 22:51:00--  https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.170.17, 128.142.53.28, 128.142.53.35, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.170.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119703858 (114M) [application/octet-stream]
Saving to: ‘SinglePhoton.hdf5’

SinglePhoton.hdf5   100%[===================>] 114.16M   248KB/s    in 3m 36s  

2025-03-25 22:54:37 (542 KB/s) - ‘SinglePhoton.hdf5’ saved [119703858/119703858]

--2025-03-25 22:54:37--  https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.170.17, 128.142.53.35, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.170.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128927319 (123M) [app

In [ ]:
def load_dataset(path):
    with h5py.File(path, "r") as f:
        X = f['X'][:]
        y = f['y'][:]
    return X, y

In [ ]:
X_p, y_p = load_dataset("SinglePhoton.hdf5")
X_e, y_e = load_dataset("SingleElectron.hdf5")

In [ ]:
X = np.concatenate([X_p[:150000], X_e[:150000]], axis=0)
y = np.concatenate([y_p[:150000], y_e[:150000]], axis=0)

In [ ]:
mean = np.mean(X, axis=(0, 1, 2))
std = np.std(X, axis=(0, 1, 2))
X = (X - mean) / std

In [ ]:
print(X.shape, y.shape)

(300000, 32, 32, 2) (300000,)


In [ ]:
X_tensor = torch.tensor(X, dtype=torch.float32).permute(0, 3, 1, 2)  # (N, C, H, W)
y_tensor = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet15(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BasicBlock(2, 16)
        self.layer2 = BasicBlock(16, 32, stride=2)
        self.layer3 = BasicBlock(32, 64, stride=2)
        self.layer4 = BasicBlock(64, 128, stride=2)
        self.layer5 = BasicBlock(128, 128)
        self.layer6 = BasicBlock(128, 128)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
model = ResNet15().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# 训练过程
for epoch in range(100):
    model.train()
    total_loss = 0
    correct = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = outputs.argmax(dim=1)
        correct += (pred == y_batch).sum().item()

    acc = correct / len(train_dataset)
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Train Acc: {acc:.4f}")

Epoch 1, Loss: 2317.6611, Train Acc: 0.6666
Epoch 2, Loss: 2167.5605, Train Acc: 0.7079
Epoch 3, Loss: 2125.6344, Train Acc: 0.7157
Epoch 4, Loss: 2105.2474, Train Acc: 0.7201
Epoch 5, Loss: 2096.4016, Train Acc: 0.7217
Epoch 6, Loss: 2088.6314, Train Acc: 0.7233
Epoch 7, Loss: 2085.0965, Train Acc: 0.7247
Epoch 8, Loss: 2080.0359, Train Acc: 0.7248
Epoch 9, Loss: 2079.0086, Train Acc: 0.7248
Epoch 10, Loss: 2072.0367, Train Acc: 0.7265
Epoch 11, Loss: 2071.2178, Train Acc: 0.7263
Epoch 12, Loss: 2072.1049, Train Acc: 0.7257
Epoch 13, Loss: 2067.3969, Train Acc: 0.7276
Epoch 14, Loss: 2067.1409, Train Acc: 0.7269
Epoch 15, Loss: 2064.2255, Train Acc: 0.7271
Epoch 16, Loss: 2063.3967, Train Acc: 0.7278
Epoch 17, Loss: 2062.7218, Train Acc: 0.7275
Epoch 18, Loss: 2061.6298, Train Acc: 0.7285
Epoch 19, Loss: 2062.4959, Train Acc: 0.7278
Epoch 20, Loss: 2059.2545, Train Acc: 0.7282
Epoch 21, Loss: 2057.2446, Train Acc: 0.7298
Epoch 22, Loss: 2055.9964, Train Acc: 0.7291
Epoch 23, Loss: 205

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        pred = outputs.argmax(dim=1)
        correct += (pred == y_batch).sum().item()

test_acc = correct / len(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 0.7272
